In [15]:
# Get all the players from Sleeper and store the pertinent information in a SQLite database
import requests
import sqlite3
import json
from sleeper import get_all_players, get_users_in_league
from manage import clean_player_data
from config import LEAGUE_LIST
LEAGUE_ID = "1234567890123456789"
SLEEPER_API = "https://api.sleeper.app/v1/players/nfl"

with open('../data/players.json', 'r') as file:
        player_json = json.load(file)

In [16]:
# Get all the players from Sleeper
players_json = get_all_players()


In [19]:
# Get all the players from Sleeper
players_json = get_all_players()
players_df = clean_player_data(players_json)

# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
players_df.to_sql('players', conn, if_exists='replace', index=True)
conn.commit()
conn.close()

# Insert the coaches into the database

In [20]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS coaches")
# Get all the coaches in each of the leagues
for league in LEAGUE_LIST:
    coaches_df = get_users_in_league(LEAGUE_LIST[league])
    coaches_df.to_sql('coaches', conn, if_exists='append', index=False)

# Update the team names if they aren't already set by combining the display name and Team like "Team" + Display Name from the row
conn.execute("UPDATE coaches SET team_name = 'Team ' + display_name WHERE team_name IS NULL")

# Close the connection
conn.close()

# Now get the drafts from every league and put these in the database